In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import csv
import xlrd
import io
from zipfile import ZipFile
from datetime import date, timedelta, time, datetime

In [2]:
today_yyyymmdd = date.today().strftime("%Y%m%d")
print(today_yyyymmdd)

20200928


import datetime 
base = datetime.date(2020,2,1)
for i in range(0,10): 
    d=base + datetime.timedelta(i)
    d=d.strftime("%Y%m%d")
    print(f"N029kakutokuruiseki{d}.csv")

from datetime import datetime, timedelta

def date_range(start_date: datetime, end_date: datetime):
    diff = (end_date - start_date).days + 1
    return (start_date + timedelta(i) for i in range(diff))

In [3]:
#pandasの表示列数を増やす
import pandas as pd
pd.set_option('display.max_columns', 100)

ディレクトリ

In [4]:
data_dir = Path(r"C:\Users\crie072\Documents\21_Online")

In [5]:
#承認月
approval_month = "6月"
approval_yymmdd = 20200630

#Input_file
#①承認リスト（当月分）
input_file_1 = "【C143】2020年6月・7月入会_7月利用実績.xlsx"

#カラム名変更
this_month = "2020年7月カード売上額合計"
pre_month = "2020年6月カード売上額合計"

#②承認リスト（前月分）
#input_file_2 = "N029（2020年5月度入会）.xlsx"

#③ デジガレ確認用リスト
input_file_3 = "20200925〆【JFRカード御中】成果データ&問い合わせ.xlsx"


#Output_file
#①オンライン承認リスト
output_file_1 = f"online_利用条件_{today_yyyymmdd}" 
print("Output①⇒",output_file_1)

#②アフィリエイト月次成果承認確定リスト
output_file_2 = f"確認済み成果データ_利用条件_{today_yyyymmdd}" 
print("Output②⇒",output_file_2)


Output①⇒ online_利用条件_20200928
Output②⇒ 確認済み成果データ_利用条件_20200928


In [ ]:
#やることメモ
#2カ月分の入会実績2シート分について、各シートごとに利用額合計を算出する
#2シート分を縦に合算し、入会実績2カ月分の利用額合計を算出し、成果クリアした会員を算出する

①承認リスト（利用実績1カ月分)

In [6]:
print(input_file_1)

【C143】2020年6月・7月入会_7月利用実績.xlsx


In [7]:
df_app = pd.read_excel(data_dir /r"input"/f"{input_file_1}",sheet_name = 1)

#シート名取得
bk = pd.ExcelFile(data_dir/r"input"/f"{input_file_1}")
print(bk.sheet_names[1])

2020年7月入会


In [8]:
df_app

,カード加入日,会員カード発行区分,続柄,カード提携会社1CD,カード提携会社2CD,カード種類CD,カードブランドCD,切替CD,勧誘CD,受付部支店,性別CD,再発行CD,旧カード提携会社1CD,旧カード提携会社2CD,旧カード種類CD,旧カードブランドCD,入会申込番号,満年齢,リボ支払方法CD,勧誘団体CD,提携カード特殊エリア,エンボス内容,カード年会費CD,リボ利用対象CD,2020年7月カード売上額合計
0,20200701,1,1,D02,4,2,2,0,3,104,2,0,NaN,NaN,NaN,NaN,155603602467338,45,12.0,4000,NaN,4.000010e+09,2,0,8239
1,20200701,1,0,D02,1,2,1,0,3,174,1,0,NaN,NaN,NaN,NaN,155603602468161,71,12.0,11000,6.291988e+12,1.100000e+04,2,0,0
2,20200701,5,0,D00,8,31,1,1,3,104,2,0,D00,50.0,31.0,0.0,155503120804691,69,12.0,4000,2.000000e+01,4.000000e+03,2,0,2526
3,20200701,1,0,D00,8,31,1,0,8,107,2,0,NaN,NaN,NaN,NaN,155203007733259,70,12.0,7000,2.000000e+01,7.000401e+09,2,0,0
4,20200701,5,0,D00,8,31,1,1,3,107,2,0,D00,50.0,31.0,0.0,155503121018085,62,12.0,7000,2.000000e+01,7.000401e+09,2,0,18200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6942,20200731,4,0,D00,1,2,1,1,4,1,2,0,D00,1.0,31.0,1.0,155603602540043,55,12.0,6000,8.020200e+09,6.000343e+09,0,0,0
6943,20200731,4,0,D00,1,2,1,1,3,1,1,0,D00,1.0,31.0,1.0,155603602540035,61,52.0,2000,NaN,2.000000e+03,0,1,0
6944,20200731,5,0,D00,1,2,1,1,3,1,1,0,D00,8.0,31.0,1.0,155603602539995,61,12.0,7000,2.000000e+01,7.000079e+09,0,0,0
6945,20200731,5,0,D00,1,2,1,1,4,1,2,0,D00,4.0,31.0,2.0,155603602539979,38,12.0,7000,8.020200e+09,7.000000e+03,0,0,0


In [9]:
df_app = df_app.rename(columns={f'{this_month}':'カード売上額合計'})

In [10]:
print(df_app.shape)
print(df_app.index)
print(df_app.columns)
print(df_app.dtypes)

(6947, 25)
RangeIndex(start=0, stop=6947, step=1)
Index(['カード加入日', '会員カード発行区分', '続柄', 'カード提携会社1CD', 'カード提携会社2CD', 'カード種類CD',
       'カードブランドCD', '切替CD', '勧誘CD', '受付部支店', '性別CD', '再発行CD', '旧カード提携会社1CD',
       '旧カード提携会社2CD', '旧カード種類CD', '旧カードブランドCD', '入会申込番号', '満年齢', 'リボ支払方法CD',
       '勧誘団体CD', '提携カード特殊エリア', 'エンボス内容', 'カード年会費CD', 'リボ利用対象CD', 'カード売上額合計'],
      dtype='object')
カード加入日           int64
会員カード発行区分       object
続柄               int64
カード提携会社1CD      object
カード提携会社2CD       int64
カード種類CD          int64
カードブランドCD        int64
切替CD             int64
勧誘CD             int64
受付部支店            int64
性別CD             int64
再発行CD            int64
旧カード提携会社1CD     object
旧カード提携会社2CD    float64
旧カード種類CD       float64
旧カードブランドCD     float64
入会申込番号           int64
満年齢              int64
リボ支払方法CD       float64
勧誘団体CD           int64
提携カード特殊エリア     float64
エンボス内容         float64
カード年会費CD         int64
リボ利用対象CD         int64
カード売上額合計         int64
dtype: object


オンライン入会客を特定

特定加入月&クレカ利用金額5000円以上に限定

In [11]:
df_app =df_app[(df_app["会員カード発行区分"] == "1") 
               & (df_app["続柄"] == 0) 
               & ~(df_app["カード提携会社2CD"] == 95) 
               & (df_app["切替CD"] == 0)
               #& (df_app["カード加入日"] <= approval_yymmdd)
               & (df_app["カード売上額合計"] >= 5000)
               & (df_app["受付部支店"] == 299)]

In [12]:
#含まれてないか確認
print(df_app["会員カード発行区分"].unique())
print(df_app["続柄"].unique())
print(df_app["カード提携会社2CD"].unique())
print(df_app["切替CD"].unique())
print(df_app["受付部支店"].unique())
df_app.shape[0]

['1']
[0]
[ 4  1  8 18 14]
[0]
[299]


203

In [13]:
#"成果"カラムを追加
df_app['成果'] = "可"
df_app['参照用'] = df_app['入会申込番号'] #"参照用"はデジガレとのマッチング用

df_app=df_app[['入会申込番号','成果','カード加入日','参照用','カード売上額合計']]
df_app

,入会申込番号,成果,カード加入日,参照用,カード売上額合計
29,155051921921736,可,20200701,155051921921736,35345
80,155051921919995,可,20200701,155051921919995,96855
106,155051921944837,可,20200701,155051921944837,9000
120,155051921944332,可,20200701,155051921944332,6897
144,155051921944142,可,20200701,155051921944142,10263
...,...,...,...,...,...
4083,155051921949026,可,20200720,155051921949026,5100
4084,155051921948911,可,20200720,155051921948911,7752
4092,155051921934507,可,20200720,155051921934507,7280
4366,155051921956849,可,20200721,155051921956849,95700


In [14]:
#df_app_m1 = df_app.drop(columns=['参照用'])
df_app_m1 = df_app
df_app_m1

,入会申込番号,成果,カード加入日,参照用,カード売上額合計
29,155051921921736,可,20200701,155051921921736,35345
80,155051921919995,可,20200701,155051921919995,96855
106,155051921944837,可,20200701,155051921944837,9000
120,155051921944332,可,20200701,155051921944332,6897
144,155051921944142,可,20200701,155051921944142,10263
...,...,...,...,...,...
4083,155051921949026,可,20200720,155051921949026,5100
4084,155051921948911,可,20200720,155051921948911,7752
4092,155051921934507,可,20200720,155051921934507,7280
4366,155051921956849,可,20200721,155051921956849,95700


CSVアウトプット

In [15]:
df_app_m1.to_csv(data_dir /r"output"/f"{output_file_1}_{bk.sheet_names[1]}.csv", index=False,header=True,encoding='shift-jis')

①-2承認リスト（利用実績2カ月分)

In [16]:
print(input_file_1)

【C143】2020年6月・7月入会_7月利用実績.xlsx


In [17]:
df_app = pd.read_excel(data_dir /r"input"/f"{input_file_1}",sheet_name = 0)

#シート名取得
bk = pd.ExcelFile(data_dir/r"input"/f"{input_file_1}")
print(bk.sheet_names[0])

2020年6月入会


In [18]:
df_app

,カード加入日,会員カード発行区分,続柄,カード提携会社1CD,カード提携会社2CD,カード種類CD,カードブランドCD,切替CD,勧誘CD,受付部支店,性別CD,再発行CD,旧カード提携会社1CD,旧カード提携会社2CD,旧カード種類CD,旧カードブランドCD,入会申込番号,満年齢,リボ支払方法CD,勧誘団体CD,提携カード特殊エリア,エンボス内容,カード年会費CD,リボ利用対象CD,2020年6月カード売上額合計,2020年7月カード売上額合計
0,20200601,4,0,D00,1,31,1,1,3,103,2,0,D00,1.0,2.0,1.0,155603602534418,59,47.0,3000,NaN,3.000000e+03,0,0,10612,10612
1,20200601,1,0,D02,1,2,1,0,3,102,1,0,NaN,NaN,NaN,NaN,155603602336889,54,12.0,2000,NaN,2.000010e+09,2,0,248,0
2,20200601,1,0,D00,8,31,1,0,8,153,2,0,NaN,NaN,NaN,NaN,155203017717409,59,12.0,53000,20.0,5.300000e+04,2,0,0,0
3,20200601,2,3,D00,1,31,1,0,3,1,2,0,NaN,NaN,NaN,NaN,155501822317608,84,47.0,3000,NaN,3.000000e+03,0,0,0,0
4,20200601,1,0,D00,95,0,0,0,3,107,1,0,NaN,NaN,NaN,NaN,155603602336723,41,NaN,0,NaN,NaN,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5274,20200630,1,0,D02,4,2,2,0,3,104,2,0,NaN,NaN,NaN,NaN,155603602466991,36,12.0,4000,NaN,4.000010e+09,2,0,0,8736
5275,20200630,1,2,D02,4,2,2,0,3,101,2,0,NaN,NaN,NaN,NaN,155603602466926,23,12.0,1000,NaN,1.000010e+09,2,0,0,5902
5276,20200630,1,0,D02,4,2,2,0,3,101,2,0,NaN,NaN,NaN,NaN,155603602466926,54,12.0,1000,NaN,1.000010e+09,2,0,0,96321
5277,20200630,1,1,D02,4,2,2,0,3,102,2,0,NaN,NaN,NaN,NaN,155603602466843,34,12.0,2000,NaN,2.000010e+09,2,0,0,0


In [19]:
df_app['カード売上額合計'] = df_app[f'{this_month}'] + df_app[f'{pre_month}'] 
#df_app = df_app.rename(columns={f'{this_month}':'カード売上額合計'})
df_app

,カード加入日,会員カード発行区分,続柄,カード提携会社1CD,カード提携会社2CD,カード種類CD,カードブランドCD,切替CD,勧誘CD,受付部支店,性別CD,再発行CD,旧カード提携会社1CD,旧カード提携会社2CD,旧カード種類CD,旧カードブランドCD,入会申込番号,満年齢,リボ支払方法CD,勧誘団体CD,提携カード特殊エリア,エンボス内容,カード年会費CD,リボ利用対象CD,2020年6月カード売上額合計,2020年7月カード売上額合計,カード売上額合計
0,20200601,4,0,D00,1,31,1,1,3,103,2,0,D00,1.0,2.0,1.0,155603602534418,59,47.0,3000,NaN,3.000000e+03,0,0,10612,10612,21224
1,20200601,1,0,D02,1,2,1,0,3,102,1,0,NaN,NaN,NaN,NaN,155603602336889,54,12.0,2000,NaN,2.000010e+09,2,0,248,0,248
2,20200601,1,0,D00,8,31,1,0,8,153,2,0,NaN,NaN,NaN,NaN,155203017717409,59,12.0,53000,20.0,5.300000e+04,2,0,0,0,0
3,20200601,2,3,D00,1,31,1,0,3,1,2,0,NaN,NaN,NaN,NaN,155501822317608,84,47.0,3000,NaN,3.000000e+03,0,0,0,0,0
4,20200601,1,0,D00,95,0,0,0,3,107,1,0,NaN,NaN,NaN,NaN,155603602336723,41,NaN,0,NaN,NaN,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5274,20200630,1,0,D02,4,2,2,0,3,104,2,0,NaN,NaN,NaN,NaN,155603602466991,36,12.0,4000,NaN,4.000010e+09,2,0,0,8736,8736
5275,20200630,1,2,D02,4,2,2,0,3,101,2,0,NaN,NaN,NaN,NaN,155603602466926,23,12.0,1000,NaN,1.000010e+09,2,0,0,5902,5902
5276,20200630,1,0,D02,4,2,2,0,3,101,2,0,NaN,NaN,NaN,NaN,155603602466926,54,12.0,1000,NaN,1.000010e+09,2,0,0,96321,96321
5277,20200630,1,1,D02,4,2,2,0,3,102,2,0,NaN,NaN,NaN,NaN,155603602466843,34,12.0,2000,NaN,2.000010e+09,2,0,0,0,0


In [20]:
print(df_app.shape)
print(df_app.index)
print(df_app.columns)
print(df_app.dtypes)

(5279, 27)
RangeIndex(start=0, stop=5279, step=1)
Index(['カード加入日', '会員カード発行区分', '続柄', 'カード提携会社1CD', 'カード提携会社2CD', 'カード種類CD',
       'カードブランドCD', '切替CD', '勧誘CD', '受付部支店', '性別CD', '再発行CD', '旧カード提携会社1CD',
       '旧カード提携会社2CD', '旧カード種類CD', '旧カードブランドCD', '入会申込番号', '満年齢', 'リボ支払方法CD',
       '勧誘団体CD', '提携カード特殊エリア', 'エンボス内容', 'カード年会費CD', 'リボ利用対象CD',
       '2020年6月カード売上額合計', '2020年7月カード売上額合計', 'カード売上額合計'],
      dtype='object')
カード加入日               int64
会員カード発行区分           object
続柄                   int64
カード提携会社1CD          object
カード提携会社2CD           int64
カード種類CD              int64
カードブランドCD            int64
切替CD                 int64
勧誘CD                 int64
受付部支店                int64
性別CD                 int64
再発行CD                int64
旧カード提携会社1CD         object
旧カード提携会社2CD        float64
旧カード種類CD           float64
旧カードブランドCD         float64
入会申込番号               int64
満年齢                  int64
リボ支払方法CD           float64
勧誘団体CD               int64
提携カード特殊エリア         float64
エンボス内容   

オンライン入会客を特定

特定加入月&クレカ利用金額5000円以上に限定

In [21]:
df_app =df_app[(df_app["会員カード発行区分"] == "1") 
               & (df_app["続柄"] == 0) 
               & ~(df_app["カード提携会社2CD"] == 95) 
               & (df_app["切替CD"] == 0)
               #& (df_app["カード加入日"] <= approval_yymmdd)
               & (df_app["カード売上額合計"] >= 5000)
               & (df_app["受付部支店"] == 299)]

In [22]:
#含まれてないか確認
print(df_app["会員カード発行区分"].unique())
print(df_app["続柄"].unique())
print(df_app["カード提携会社2CD"].unique())
print(df_app["切替CD"].unique())
print(df_app["受付部支店"].unique())
df_app.shape[0]

['1']
[0]
[ 4  1  8 11 14]
[0]
[299]


215

In [23]:
#"成果"カラムを追加
df_app['成果'] = "可"
df_app['参照用'] = df_app['入会申込番号'] #"参照用"はデジガレとのマッチング用

df_app=df_app[['入会申込番号','成果','カード加入日','参照用','カード売上額合計']]

C:\Users\crie072\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\crie072\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [24]:
#df_app_m2 = df_app.drop(columns=['参照用'])
df_app_m2 = df_app
df_app_m2

,入会申込番号,成果,カード加入日,参照用,カード売上額合計
15,155051921883902,可,20200601,155051921883902,5389
16,155051921872202,可,20200601,155051921872202,38900
20,155051921900268,可,20200601,155051921900268,32923
54,155051921903262,可,20200601,155051921903262,600000
104,155051921870925,可,20200601,155051921870925,5463
...,...,...,...,...,...
5208,155051921941551,可,20200630,155051921941551,31900
5213,155051921935314,可,20200630,155051921935314,12217
5214,155051921922148,可,20200630,155051921922148,8216
5216,155051921919631,可,20200630,155051921919631,32127


CSVアウトプット

In [25]:
df_app_m2.to_csv(data_dir /r"output"/f"{output_file_1}_{bk.sheet_names[0]}.csv", index=False,header=True,encoding='shift-jis')

2カ月入会月分合算

In [26]:
df_app_concat = pd.concat([df_app_m1,df_app_m2])
df_app_concat

,入会申込番号,成果,カード加入日,参照用,カード売上額合計
29,155051921921736,可,20200701,155051921921736,35345
80,155051921919995,可,20200701,155051921919995,96855
106,155051921944837,可,20200701,155051921944837,9000
120,155051921944332,可,20200701,155051921944332,6897
144,155051921944142,可,20200701,155051921944142,10263
...,...,...,...,...,...
5208,155051921941551,可,20200630,155051921941551,31900
5213,155051921935314,可,20200630,155051921935314,12217
5214,155051921922148,可,20200630,155051921922148,8216
5216,155051921919631,可,20200630,155051921919631,32127


② デジガレ確認用リスト ※利用条件シート

In [27]:
print(input_file_3)

df_dg = pd.read_excel(data_dir/r"input"/f"{input_file_3}", sheet_name = 2)
#シート名取得
bk = pd.ExcelFile(data_dir/r"input"/f"{input_file_3}")

df_dg.head()

20200925〆【JFRカード御中】成果データ&問い合わせ.xlsx


,ASID,メディア名,媒体チャネル,クリック,申込,参照用,ASP名,単価,成果,カード発行日,カード売上額合計
0,21.0,ポイントインカム,ポイント,2020-06-01 01:25:55,2020-06-01 01:34:29,155051921904849,BGT,9000,保留,保留,1.550519e+14
1,128.0,Warau.jp,ポイント,2020-06-01 09:42:13,2020-06-01 09:50:12,155051921904930,BGT,9000,保留,保留,1.550519e+14
2,5954.0,GetMoney!(JIPC参加企業)（連携用）,ポイント,2020-06-01 12:35:04,2020-06-01 12:58:12,155051921905143,BGT,9000,保留,保留,1.550519e+14
3,57.0,ライフメディア,ポイント,2020-06-01 16:52:21,2020-06-01 17:03:12,155051921905432,BGT,9000,保留,20200624,1.550519e+14
4,21.0,ポイントインカム,ポイント,2020-06-01 22:16:51,2020-06-01 22:26:12,155051921905846,BGT,9000,保留,保留,1.550519e+14


In [28]:
print(bk.sheet_names[2])

成果データ（5月～利用条件追加媒体）


In [29]:
df_dg.shape[0]

918

In [30]:
#余計なカラム削除
#df_dg = df_dg.drop(columns=['Unnamed: 9','成果'])
df_dg = df_dg.drop(columns=['成果','カード発行日'])
print(df_dg.dtypes)

ASID               float64
メディア名               object
媒体チャネル              object
クリック        datetime64[ns]
申込          datetime64[ns]
参照用                  int64
ASP名                object
単価                   int64
カード売上額合計           float64
dtype: object


In [32]:
#ASIDの整数化
for i in range(len(df_dg)):
    if pd.isnull(df_dg['ASID'][i]) == True:
        df_dg.loc[:,'ASID'][i] = 0
df_dg = df_dg.astype({'ASID':int})

print(df_dg.shape)
print(df_dg.index)
print(df_dg.columns)
print(df_dg.dtypes)

(918, 9)
RangeIndex(start=0, stop=918, step=1)
Index(['ASID', 'メディア名', '媒体チャネル', 'クリック', '申込', '参照用', 'ASP名', '単価',
       'カード売上額合計'],
      dtype='object')
ASID                 int32
メディア名               object
媒体チャネル              object
クリック        datetime64[ns]
申込          datetime64[ns]
参照用                  int64
ASP名                object
単価                   int64
カード売上額合計           float64
dtype: object


In [33]:
#"申込年月"のカラム生成
df_dg['申込年'] = df_dg['申込'].dt.year
df_dg['申込月'] = df_dg['申込'].dt.month

df_dg['申込年月'] = df_dg[['申込年', '申込月']].apply(lambda x: '{}年{}月'.format(x[0], x[1]), axis=1)
df_dg = df_dg.drop(columns=['申込年','申込月'])

df_dg.head()

,ASID,メディア名,媒体チャネル,クリック,申込,参照用,ASP名,単価,カード売上額合計,申込年月
0,21,ポイントインカム,ポイント,2020-06-01 01:25:55,2020-06-01 01:34:29,155051921904849,BGT,9000,1.550519e+14,2020年6月
1,128,Warau.jp,ポイント,2020-06-01 09:42:13,2020-06-01 09:50:12,155051921904930,BGT,9000,1.550519e+14,2020年6月
2,5954,GetMoney!(JIPC参加企業)（連携用）,ポイント,2020-06-01 12:35:04,2020-06-01 12:58:12,155051921905143,BGT,9000,1.550519e+14,2020年6月
3,57,ライフメディア,ポイント,2020-06-01 16:52:21,2020-06-01 17:03:12,155051921905432,BGT,9000,1.550519e+14,2020年6月
4,21,ポイントインカム,ポイント,2020-06-01 22:16:51,2020-06-01 22:26:12,155051921905846,BGT,9000,1.550519e+14,2020年6月


In [34]:
df_app_concat

,入会申込番号,成果,カード加入日,参照用,カード売上額合計
29,155051921921736,可,20200701,155051921921736,35345
80,155051921919995,可,20200701,155051921919995,96855
106,155051921944837,可,20200701,155051921944837,9000
120,155051921944332,可,20200701,155051921944332,6897
144,155051921944142,可,20200701,155051921944142,10263
...,...,...,...,...,...
5208,155051921941551,可,20200630,155051921941551,31900
5213,155051921935314,可,20200630,155051921935314,12217
5214,155051921922148,可,20200630,155051921922148,8216
5216,155051921919631,可,20200630,155051921919631,32127


In [35]:
print(df_app_concat.dtypes)

入会申込番号       int64
成果          object
カード加入日       int64
参照用          int64
カード売上額合計     int64
dtype: object


承認データ（利用条件）とデジガレリストをマージ

In [36]:
#承認データ（df_app）をマージする前に、df_app余計なカラムを削除
df_app_tmp = df_app_concat.drop(columns=['入会申込番号'])

#承認データの"成果"をマージ
df_dg2 = pd.merge(df_dg,df_app_tmp, on =['参照用'], how = 'left')
df_dg2[df_dg2['成果'] == '可'].head()

,ASID,メディア名,媒体チャネル,クリック,申込,参照用,ASP名,単価,カード売上額合計_x,申込年月,成果,カード加入日,カード売上額合計_y
30,128,Warau.jp,ポイント,2020-06-09 01:28:03,2020-06-09 01:38:07,155051921913089,BGT,9000,1.550519e+14,2020年6月,可,20200622.0,30220.0
35,128,Warau.jp,ポイント,2020-06-09 21:05:15,2020-06-09 21:24:06,155051921914020,BGT,9000,1.550519e+14,2020年6月,可,20200625.0,6329.0
56,128,Warau.jp,ポイント,2020-06-14 22:27:52,2020-06-14 23:02:11,155051921919631,BGT,9000,1.550519e+14,2020年6月,可,20200630.0,32127.0
79,37,げん玉,ポイント,2020-06-16 21:41:40,2020-06-16 21:55:13,155051921921736,BGT,9000,1.550519e+14,2020年6月,可,20200701.0,35345.0
82,21,ポイントインカム,ポイント,2020-06-17 10:52:21,2020-06-17 10:59:08,155051921922148,BGT,9000,1.550519e+14,2020年6月,可,20200630.0,8216.0


In [37]:
df_dg2 = df_dg2.copy()

In [39]:
#マージしたらカード加入日がfloat型になってしまったため、整数型に戻す
for i in range(len(df_dg2)):
    if pd.isnull(df_dg2['カード加入日'][i]) == True:
        df_dg2.loc[:,'カード加入日'][i] = 0
        
df_dg2 = df_dg2.astype({'カード加入日':int})

In [40]:
df_dg2[df_dg2['カード加入日'] == 0].head()

,ASID,メディア名,媒体チャネル,クリック,申込,参照用,ASP名,単価,カード売上額合計_x,申込年月,成果,カード加入日,カード売上額合計_y
0,21,ポイントインカム,ポイント,2020-06-01 01:25:55,2020-06-01 01:34:29,155051921904849,BGT,9000,1.550519e+14,2020年6月,NaN,0,NaN
1,128,Warau.jp,ポイント,2020-06-01 09:42:13,2020-06-01 09:50:12,155051921904930,BGT,9000,1.550519e+14,2020年6月,NaN,0,NaN
2,5954,GetMoney!(JIPC参加企業)（連携用）,ポイント,2020-06-01 12:35:04,2020-06-01 12:58:12,155051921905143,BGT,9000,1.550519e+14,2020年6月,NaN,0,NaN
3,57,ライフメディア,ポイント,2020-06-01 16:52:21,2020-06-01 17:03:12,155051921905432,BGT,9000,1.550519e+14,2020年6月,NaN,0,NaN
4,21,ポイントインカム,ポイント,2020-06-01 22:16:51,2020-06-01 22:26:12,155051921905846,BGT,9000,1.550519e+14,2020年6月,NaN,0,NaN


In [41]:
#"成果"がNanのものは[保留]に置換する
for i in range(len(df_dg2)):
    if pd.isnull(df_dg2['成果'][i]) == True:
        df_dg2.loc[:,'成果'][i] = "保留"

C:\Users\crie072\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [42]:
#カード加入日が0のものは[保留]に置換する
for i in range(len(df_dg2)):
    if df_dg2['カード加入日'][i] == 0:
        df_dg2.loc[:,'カード加入日'][i] = "保留"

C:\Users\crie072\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\crie072\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [43]:
df_dg2

,ASID,メディア名,媒体チャネル,クリック,申込,参照用,ASP名,単価,カード売上額合計_x,申込年月,成果,カード加入日,カード売上額合計_y
0,21,ポイントインカム,ポイント,2020-06-01 01:25:55,2020-06-01 01:34:29,155051921904849,BGT,9000,1.550519e+14,2020年6月,保留,保留,NaN
1,128,Warau.jp,ポイント,2020-06-01 09:42:13,2020-06-01 09:50:12,155051921904930,BGT,9000,1.550519e+14,2020年6月,保留,保留,NaN
2,5954,GetMoney!(JIPC参加企業)（連携用）,ポイント,2020-06-01 12:35:04,2020-06-01 12:58:12,155051921905143,BGT,9000,1.550519e+14,2020年6月,保留,保留,NaN
3,57,ライフメディア,ポイント,2020-06-01 16:52:21,2020-06-01 17:03:12,155051921905432,BGT,9000,1.550519e+14,2020年6月,保留,保留,NaN
4,21,ポイントインカム,ポイント,2020-06-01 22:16:51,2020-06-01 22:26:12,155051921905846,BGT,9000,1.550519e+14,2020年6月,保留,保留,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
913,37,げん玉,ポイント,2020-08-31 13:18:32,2020-08-31 13:26:06,155051922004003,BGT,9000,NaN,2020年8月,保留,保留,NaN
914,0,getmoney,ポイント,2020-07-25 00:00:00,2020-07-25 00:00:00,155051921974289,BGT,9000,NaN,2020年7月,保留,保留,NaN
915,0,ちょびリッチ,ポイント,2020-07-07 00:00:00,2020-07-07 00:00:00,155051921955825,BGT,9000,NaN,2020年7月,可,20200710,5846.0
916,0,ポイントインカム,ポイント,2020-07-24 20:09:00,2020-07-24 00:00:00,155051921973299,BGT,9000,NaN,2020年7月,保留,保留,NaN


集計

In [44]:
#客数
df_dg2[df_dg2['成果'] == '可'].shape[0]

134

In [45]:
pd.crosstab(df_dg2['申込年月'],df_dg2['成果'])

成果,保留,可
申込年月,,
2020年6月,170,13
2020年7月,406,121
2020年8月,208,0


In [46]:
#CSVアウトプット
df_dg2.to_csv(data_dir /r"output"/f"{output_file_2}_{bk.sheet_names[2]}.csv", index=False,header=True,encoding='shift-jis')

③ デジガレ確認用リスト ※利用条件シート　先月の成果報告が正しくなかったため再度確認

In [47]:
print(input_file_3)

df_dg = pd.read_excel(data_dir/r"input"/f"{input_file_3}", sheet_name = 3)
df_dg.head()

20200925〆【JFRカード御中】成果データ&問い合わせ.xlsx


,ASID,メディア名,媒体チャネル,クリック,申込,参照用,ASP名,単価,成果,カード発行日,カード売上額合計
0,37,げん玉,ポイント,2020-05-01 00:31:25,2020-05-01 00:52:05,155051921869372,BGT,9000,保留,20200615,NaN
1,21,ポイントインカム,ポイント,2020-05-01 06:15:50,2020-05-01 06:25:05,155051921869414,BGT,9000,保留,20200611,NaN
2,21,ポイントインカム,ポイント,2020-05-01 06:51:10,2020-05-01 07:03:03,155051921869455,BGT,9000,保留,20200525,NaN
3,21,ポイントインカム,ポイント,2020-05-01 07:53:11,2020-05-01 08:06:04,155051921869497,BGT,9000,保留,20200525,NaN
4,37,げん玉,ポイント,2020-05-01 07:59:02,2020-05-01 08:25:06,155051921869513,BGT,9000,保留,20200526,NaN


In [48]:
print(bk.sheet_names[3])

成果データ（5月～利用条件追加媒体）_承認返却済み


In [49]:
df_dg.shape[0]

250

In [50]:
#余計なカラム削除
#df_dg = df_dg.drop(columns=['Unnamed: 9','成果'])
df_dg = df_dg.drop(columns=['成果','カード発行日','カード売上額合計'])
print(df_dg.dtypes)

ASID               int64
メディア名             object
媒体チャネル            object
クリック      datetime64[ns]
申込        datetime64[ns]
参照用                int64
ASP名              object
単価                 int64
dtype: object


In [51]:
#ASIDの整数化
for i in range(len(df_dg)):
    if pd.isnull(df_dg['ASID'][i]) == True:
        df_dg.loc[:,'ASID'][i] = 0
df_dg = df_dg.astype({'ASID':int})

print(df_dg.shape)
print(df_dg.index)
print(df_dg.columns)
print(df_dg.dtypes)

(250, 8)
RangeIndex(start=0, stop=250, step=1)
Index(['ASID', 'メディア名', '媒体チャネル', 'クリック', '申込', '参照用', 'ASP名', '単価'], dtype='object')
ASID               int32
メディア名             object
媒体チャネル            object
クリック      datetime64[ns]
申込        datetime64[ns]
参照用                int64
ASP名              object
単価                 int64
dtype: object


In [52]:
#"申込年月"のカラム生成
df_dg['申込年'] = df_dg['申込'].dt.year
df_dg['申込月'] = df_dg['申込'].dt.month

df_dg['申込年月'] = df_dg[['申込年', '申込月']].apply(lambda x: '{}年{}月'.format(x[0], x[1]), axis=1)
df_dg = df_dg.drop(columns=['申込年','申込月'])

df_dg.head()

,ASID,メディア名,媒体チャネル,クリック,申込,参照用,ASP名,単価,申込年月
0,37,げん玉,ポイント,2020-05-01 00:31:25,2020-05-01 00:52:05,155051921869372,BGT,9000,2020年5月
1,21,ポイントインカム,ポイント,2020-05-01 06:15:50,2020-05-01 06:25:05,155051921869414,BGT,9000,2020年5月
2,21,ポイントインカム,ポイント,2020-05-01 06:51:10,2020-05-01 07:03:03,155051921869455,BGT,9000,2020年5月
3,21,ポイントインカム,ポイント,2020-05-01 07:53:11,2020-05-01 08:06:04,155051921869497,BGT,9000,2020年5月
4,37,げん玉,ポイント,2020-05-01 07:59:02,2020-05-01 08:25:06,155051921869513,BGT,9000,2020年5月


In [53]:
df_app_concat

,入会申込番号,成果,カード加入日,参照用,カード売上額合計
29,155051921921736,可,20200701,155051921921736,35345
80,155051921919995,可,20200701,155051921919995,96855
106,155051921944837,可,20200701,155051921944837,9000
120,155051921944332,可,20200701,155051921944332,6897
144,155051921944142,可,20200701,155051921944142,10263
...,...,...,...,...,...
5208,155051921941551,可,20200630,155051921941551,31900
5213,155051921935314,可,20200630,155051921935314,12217
5214,155051921922148,可,20200630,155051921922148,8216
5216,155051921919631,可,20200630,155051921919631,32127


In [54]:
print(df_app_concat.dtypes)

入会申込番号       int64
成果          object
カード加入日       int64
参照用          int64
カード売上額合計     int64
dtype: object


In [55]:
#承認データ（df_app）をマージする前に、df_app余計なカラムを削除
df_app_tmp = df_app_concat.drop(columns=['入会申込番号'])

#承認データの"成果"をマージ
df_dg2 = pd.merge(df_dg,df_app_tmp, on =['参照用'], how = 'left')
df_dg2[df_dg2['成果'] == '可'].head()

,ASID,メディア名,媒体チャネル,クリック,申込,参照用,ASP名,単価,申込年月,成果,カード加入日,カード売上額合計
1,21,ポイントインカム,ポイント,2020-05-01 06:15:50,2020-05-01 06:25:05,155051921869414,BGT,9000,2020年5月,可,20200611.0,11381.0
6,37,げん玉,ポイント,2020-05-01 10:59:02,2020-05-01 11:14:05,155051921869646,BGT,9000,2020年5月,可,20200616.0,6701.0
12,37,げん玉,ポイント,2020-05-01 17:19:07,2020-05-01 17:39:04,155051921870339,BGT,9000,2020年5月,可,20200602.0,18394.0
16,37,げん玉,ポイント,2020-05-01 22:59:04,2020-05-01 23:11:05,155051921870743,BGT,9000,2020年5月,可,20200602.0,6127.0
20,21,ポイントインカム,ポイント,2020-05-02 06:44:05,2020-05-02 06:56:05,155051921870925,BGT,9000,2020年5月,可,20200601.0,5463.0


In [56]:
df_dg2 = df_dg2.copy()

In [58]:
#マージしたらカード加入日がfloat型になってしまったため、整数型に戻す
for i in range(len(df_dg2)):
    if pd.isnull(df_dg2['カード加入日'][i]) == True:
        df_dg2.loc[:,'カード加入日'][i] = 0
        
df_dg2 = df_dg2.astype({'カード加入日':int})

In [60]:
df_dg2[df_dg2['カード加入日'] == 0].head()

,ASID,メディア名,媒体チャネル,クリック,申込,参照用,ASP名,単価,申込年月,成果,カード加入日,カード売上額合計
0,37,げん玉,ポイント,2020-05-01 00:31:25,2020-05-01 00:52:05,155051921869372,BGT,9000,2020年5月,NaN,0,NaN
2,21,ポイントインカム,ポイント,2020-05-01 06:51:10,2020-05-01 07:03:03,155051921869455,BGT,9000,2020年5月,NaN,0,NaN
3,21,ポイントインカム,ポイント,2020-05-01 07:53:11,2020-05-01 08:06:04,155051921869497,BGT,9000,2020年5月,NaN,0,NaN
4,37,げん玉,ポイント,2020-05-01 07:59:02,2020-05-01 08:25:06,155051921869513,BGT,9000,2020年5月,NaN,0,NaN
5,21,ポイントインカム,ポイント,2020-05-01 10:27:46,2020-05-01 10:37:05,155051921869588,BGT,9000,2020年5月,NaN,0,NaN


In [62]:
#"成果"がNanのものは[保留]に置換する
for i in range(len(df_dg2)):
    if pd.isnull(df_dg2['成果'][i]) == True:
        df_dg2.loc[:,'成果'][i] = "保留"
        
#カード加入日が0のものは[保留]に置換する
for i in range(len(df_dg2)):
    if df_dg2['カード加入日'][i] == 0:
        df_dg2.loc[:,'カード加入日'][i] = "保留"

In [63]:
df_dg2.head()

,ASID,メディア名,媒体チャネル,クリック,申込,参照用,ASP名,単価,申込年月,成果,カード加入日,カード売上額合計
0,37,げん玉,ポイント,2020-05-01 00:31:25,2020-05-01 00:52:05,155051921869372,BGT,9000,2020年5月,保留,保留,NaN
1,21,ポイントインカム,ポイント,2020-05-01 06:15:50,2020-05-01 06:25:05,155051921869414,BGT,9000,2020年5月,可,20200611,11381.0
2,21,ポイントインカム,ポイント,2020-05-01 06:51:10,2020-05-01 07:03:03,155051921869455,BGT,9000,2020年5月,保留,保留,NaN
3,21,ポイントインカム,ポイント,2020-05-01 07:53:11,2020-05-01 08:06:04,155051921869497,BGT,9000,2020年5月,保留,保留,NaN
4,37,げん玉,ポイント,2020-05-01 07:59:02,2020-05-01 08:25:06,155051921869513,BGT,9000,2020年5月,保留,保留,NaN


集計

In [64]:
#客数
df_dg2[df_dg2['成果'] == '可'].shape[0]

57

In [65]:
pd.crosstab(df_dg2['申込年月'],df_dg2['成果'])

成果,保留,可
申込年月,,
2020年5月,147,49
2020年6月,46,8


In [70]:
#CSVアウトプット
df_dg2.to_csv(data_dir /r"output"/f"{output_file_2}_{bk.sheet_names[3]}.csv", index=False,header=True,encoding='shift-jis')

カード利用条件ありにしかカード加入日が入力されていないので、再付与

In [66]:
print(input_file_1)

df_new1 = pd.read_excel(data_dir /r"input"/f"{input_file_1}",usecols =['カード加入日','入会申込番号'],sheet_name=0)
df_new2 = pd.read_excel(data_dir /r"input"/f"{input_file_1}",usecols =['カード加入日','入会申込番号'],sheet_name=1)

df_new = pd.concat([df_new1,df_new2])

df_new = df_new.rename(columns={'入会申込番号':'参照用'}) #"参照用"はデジガレとのマッチング用
df_new

【C143】2020年6月・7月入会_7月利用実績.xlsx


,カード加入日,参照用
0,20200601,155603602534418
1,20200601,155603602336889
2,20200601,155203017717409
3,20200601,155501822317608
4,20200601,155603602336723
...,...,...
6942,20200731,155603602540043
6943,20200731,155603602540035
6944,20200731,155603602539995
6945,20200731,155603602539979


In [67]:
df_dg3 = df_dg2.drop(columns={'カード加入日'})
df_dg3 = pd.merge(df_dg3,df_new, on =['参照用'], how = 'left')

#マージしたらカード加入日がfloat型になってしまったため、整数型に戻す
for i in range(len(df_dg3)):
    if pd.isnull(df_dg3['カード加入日'][i]) == True:
        df_dg3.loc[:,'カード加入日'][i] = 0
        
df_dg3 = df_dg3.astype({'カード加入日':int})

       
#カード加入日が0のものは[保留]に置換する
for i in range(len(df_dg3)):
    if df_dg3['カード加入日'][i] == 0:
        df_dg3.loc[:,'カード加入日'][i] = "保留"

C:\Users\crie072\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\crie072\anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
C:\Users\crie072\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, 

In [68]:
df_dg3.head()

,ASID,メディア名,媒体チャネル,クリック,申込,参照用,ASP名,単価,申込年月,成果,カード売上額合計,カード加入日
0,37,げん玉,ポイント,2020-05-01 00:31:25,2020-05-01 00:52:05,155051921869372,BGT,9000,2020年5月,保留,NaN,20200615
1,21,ポイントインカム,ポイント,2020-05-01 06:15:50,2020-05-01 06:25:05,155051921869414,BGT,9000,2020年5月,可,11381.0,20200611
2,21,ポイントインカム,ポイント,2020-05-01 06:51:10,2020-05-01 07:03:03,155051921869455,BGT,9000,2020年5月,保留,NaN,保留
3,21,ポイントインカム,ポイント,2020-05-01 07:53:11,2020-05-01 08:06:04,155051921869497,BGT,9000,2020年5月,保留,NaN,保留
4,37,げん玉,ポイント,2020-05-01 07:59:02,2020-05-01 08:25:06,155051921869513,BGT,9000,2020年5月,保留,NaN,保留


In [69]:
#CSVアウトプット
df_dg3.to_csv(data_dir /r"output"/f"{output_file_2}_{bk.sheet_names[3]}.csv", index=False,header=True,encoding='shift-jis')